#### Code to train models

In [64]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
cd ../src

/home/theo/kaggle/siim_covid/src


## Imports

In [66]:
import os
import cv2
import json
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter
from tqdm.notebook import tqdm

In [67]:
from params import *

In [68]:
from data.preparation import prepare_dataframe, handle_duplicates
from data.dataset import CovidDetDataset, CovidClsDataset
from data.transforms import get_transfos_det, get_transfos_cls

from model_zoo.models import get_model

from utils.plot import plot_sample

from utils.logger import Config

from utils.metrics import per_class_average_precision_score, study_level_map

## Data

### Load

In [69]:
EXP_FOLDER = LOG_PATH + "2021-07-24/1/"
EXP_FOLDER = LOG_PATH + "2021-07-24/2/"
EXP_FOLDER = LOG_PATH + "2021-07-25/0/"

In [70]:
config = Config(json.load(open(EXP_FOLDER + "config.json", 'r')))

In [71]:
# metrics = pd.read_csv(EXP_FOLDER + "metrics.csv")
pred_oof_img = np.load(EXP_FOLDER + "pred_oof_img.npy")
pred_oof_study = np.load(EXP_FOLDER + "pred_oof_study.npy")

df = pd.read_csv(EXP_FOLDER + "data.csv")

In [72]:
FOLD_IDX = 0

In [73]:
weights = sorted([EXP_FOLDER + p for p in os.listdir(EXP_FOLDER) if 'pt' in p])

In [74]:
# model = get_model(
#     config.selected_model,
#     reduce_stride=config.reduce_stride,
#     num_classes=config.num_classes,
# ).to(config.device)
# model.zero_grad()

# model.eval()

# model = load_model_weights(model, weights[FOLD_IDX])

In [75]:
pred_cols = [c + "_pred" for c in CLASSES]
df[pred_cols] = pred_oof_study
# df = df[df['negative_pred'] > 0]

In [76]:
# df_study = df[['study_id'] + pred_cols + CLASSES].groupby('study_id').agg(np.mean)
# per_class_average_precision_score(df_study[pred_cols].values, df_study[CLASSES].values, num_classes=NUM_CLASSES) * 2/3

In [77]:
study_map = study_level_map(df[pred_cols].values, df[CLASSES].values, df['study_id'].values)
print(f'Study mAP : {study_map :.3f}')

Study mAP : 0.350


## Results

In [78]:
pred_oof_study = pd.read_csv('../output/OOF_study_only_EBV2M_768.csv')
pred_oof_study['id'] = pred_oof_study['id'].apply(lambda x: x.split('_')[0])

df_study = df[['study_id'] + pred_cols + CLASSES].groupby('study_id').agg(np.mean)
df_study = df_study.merge(pred_oof_study, how="left", left_on="study_id", right_on="id").dropna()

In [79]:
def proc(x):
    x = x.split('0 0 1 1')[:4]
    x = [float(y.strip().split(' ')[1]) for y in x]
    return x

pred_oof_study = np.array(df_study['PredictionString'].apply(proc).values.tolist())
# pred_oof_study = np.random.random(pred_oof_study.shape)
df_study[pred_cols] = pred_oof_study

In [80]:
per_class_average_precision_score(
    df_study[pred_cols].values,
    df_study[CLASSES].values, 
num_classes=NUM_CLASSES) * 2/3

0.3781226084618286